In [ ]:
cd ../../../
./aestas.ps1 build

In [1]:
#load "privateVar.fsx"
// use a individual file to store your private data, like:
// #r "C:\\Users\\YourUserName\\Source\\Aestas\\bin\\Debug\\net8.0\\aestas.dll"
// let geminiKey = "Your Api Key"
// let name = "Bot Name"
// let systemInstruction = "Your System Instruction"
open PrivateVar
open Aestas
open Aestas.Prim
open Aestas.Core
open Aestas.Core.AestasBot
open Aestas.AutoInit
open Aestas.Llms.Gemini

In [ ]:
// open Aestas.Misc.SiliconFlow
// let (Ok img) =
//     textToImageMethod 
//         "https://api.siliconflow.cn/v1/black-forest-labs/FLUX.1-schnell/text-to-image" 
//         {key = "key"} 
//         {
//             prompt = "scenery of a beautiful village in the mountains"
//             negative = ""
//             resolution = 1024, 1024
//             seed = None
//         }
//     |> Async.RunSynchronously
// DisplayExtensions.DisplayAs(Convert.ToBase64String img,"image/png")

In [2]:
open Aestas.Core.Builtin
let mutable display = ""
let virtualDomain = VirtualDomain((fun mid s -> display <- s.ToString()), ignore, {
        uid = 1u
        name = name
    },{
        uid = 0u
        name = "User"
    }, 0u, "Test", true)

In [3]:

// seq {InitTypes.Command} |> set |> AutoInit.init true
let model = GeminiLlm({
    apiKey = Some geminiKey
    gcloudPath = None
    safetySettings = [|
        {category = "HARM_CATEGORY_HARASSMENT"; threshold = "BLOCK_NONE"}
        {category = "HARM_CATEGORY_HATE_SPEECH"; threshold = "BLOCK_NONE"}
        {category = "HARM_CATEGORY_SEXUALLY_EXPLICIT"; threshold = "BLOCK_NONE"}
        {category = "HARM_CATEGORY_DANGEROUS_CONTENT"; threshold = "BLOCK_NONE"}
    |]
    generation_configs = {
        defaultGenerationConfig with
            temperature = Some 1.
            maxLength = Some 4096
            topK = Some 64
            topP = Some 1.
    } |> Some
    }, 
    "gemini-1.5-flash-exp-0827")
let bot = 
    createBot {|
        name = name
        model = model
        systemInstruction = systemInstruction |> Some
        systemInstructionBuilder = Builtin.buildSystemInstruction |> PipeLineChain |> Some
        friendStrategy = StrategyFriendAll |> Some
        contentLoadStrategy = StrategyLoadOnlyMentionedOrPrivate |> Some
        contentParseStrategy = None
        messageReplyStrategy = StrategyReplyOnlyMentionedOrPrivate |> Some
        messageCacheStrategy = None
        contextStrategy = StrategyContextTrimWhenExceedLength 300 |> Some
        inputPrefixBuilder = Builtin.buildPrefix |> Some
        userCommandPrivilege = None
    |}
bindModel bot model

In [ ]:
bindDomain bot virtualDomain
virtualDomain.Bot

In [ ]:
bot.CommandExecuters.Keys

In [ ]:
virtualDomain.Input([AestasText "\\gemini:help"]) |> Async.RunSynchronously |> ignore
printfn "[%s]" display
printfn "|%A" virtualDomain.Messages

In [ ]:
virtualDomain.Input([AestasText "hello"]) |> Async.RunSynchronously |> ignore
printfn "[%s]" display
printfn "|%A" virtualDomain.Messages

In [ ]:
virtualDomain.Input([AestasText "\\gemini:dump"]) |> Async.RunSynchronously |> ignore
printfn "[%s]" display
printfn "|%A" virtualDomain.Messages

In [3]:
let t = "a"

In [9]:
bot.ClearCachedContext virtualDomain

In [ ]:
Logger.getLogs(0) |> Seq.iter (fun x -> printfn "%A" x)